In [3]:
%pip install -U "ray[client]==2.9.0"
%pip install -U "ray[serve]==2.9.0"
%pip install starlette
%pip install transformers
%pip install -U ipywidgets

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.5 MB/s eta 0:00:002.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.0 MB/s eta 0:00:001.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached blessed-1.20.0-py2.py3-none-any.whl (58 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 2.4 MB/s eta 0:00:002.4 MB/s eta 0:00:01
  Created wheel for gpustat: filename=gpustat-1.1.1-py3-none-any.whl size=26537 sha256=744091a25bb16b7d8e4614d2c2a54d3dc980705672830cb8cc0

In [4]:
from starlette.requests import Request

import ray
from ray import serve

from transformers import pipeline


@serve.deployment(num_replicas=2, ray_actor_options={"num_cpus": 0.2, "num_gpus": 0})
class Translator:
    def __init__(self):
        # Load model
        self.model = pipeline("translation_en_to_fr", model="t5-small")

    def translate(self, text: str) -> str:
        # Run inference
        model_output = self.model(text)

        # Post-process output to return only the translation text
        translation = model_output[0]["translation_text"]

        return translation

    async def __call__(self, http_request: Request) -> str:
        english_text: str = await http_request.json()
        return self.translate(english_text)


translator_app = Translator.bind()

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
!serve run serve_quickstart:translator_app

2024-11-12 23:35:59,198	INFO scripts.py:438 -- Running import path: 'serve_quickstart:translator_app'.
Traceback (most recent call last):
  File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/bin/serve", line 8, in <module>
    sys.exit(cli())
  File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/lib/python3.8/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/lib/python3.8/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/lib/python3.8/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/lib/python3.8/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/lib/python3.8/site-packages/click/core.py", l